# Agentic AI based Portfolio Manager
## Powered by IBM Watsonx using Langgraph supervisor and agents
##### Thanks to Adarsh Balan @ Analytics Vidya https://www.analyticsvidhya.com/blog/2025/02/financial-market-analysis-ai-agent/

In [1]:
#imports

import os
from dotenv import load_dotenv



## Connect to watsonx

In [2]:
#### Connect to watsonx
from langchain_ibm import ChatWatsonx
load_dotenv()
# Required
WATSONX_URL=os.getenv("WATSONX_URL")
WATSONX_API_KEY=os.getenv("WATSONX_API_KEY")
WATSONX_APIKEY=os.getenv("WATSONX_API_KEY")
WX_API_KEY = os.getenv("WATSONX_API_KEY")
WATSONX_PROJECT_ID=os.getenv("PROJECT_ID")

parameters = {
    "temperature": 0.9,
    "max_tokens": 200,
}

model = ChatWatsonx(
    model_id="ibm/granite-3-8b-instruct",
    url="https://us-south.ml.cloud.ibm.com",
    project_id=WATSONX_PROJECT_ID,
    params=parameters,
)



## Write the tools needed Fetch market data, analyse sentiments, compute quant metrics, decide investment strategy

In [3]:
# 1. Fetching Market Data
def fetch_market_data(stock_symbol: str) -> dict:
    """Simulate fetching stock market data for a given symbol."""
    market_data = {
        "AAPL": {"price": 185.22, "pe_ratio": 28.3, "eps": 6.5, "revenue_growth": 8.5},
        "GOOG": {"price": 142.11, "pe_ratio": 26.1, "eps": 5.8, "revenue_growth": 7.9},
        "TSLA": {"price": 220.34, "pe_ratio": 40.2, "eps": 3.5, "revenue_growth": 6.2},
    }
    return market_data.get(stock_symbol, {})

In [4]:
# 2. Sentiment Analysis
def analyze_sentiment(stock_symbol: str) -> dict:
    """Perform sentiment analysis on financial news for a stock."""
    sentiment_scores = {
        "AAPL": {"news_sentiment": "Positive", "social_sentiment": "Neutral"},
        "GOOG": {"news_sentiment": "Negative", "social_sentiment": "Positive"},
        "TSLA": {"news_sentiment": "Positive", "social_sentiment": "Negative"},
    }
    return sentiment_scores.get(stock_symbol, {})

In [5]:
# 3. Quantitative Analysis
def compute_quant_metrics(stock_symbol: str) -> dict:
    """Compute SMA, EMA, and volatility for stock."""
    quant_metrics = {
        "AAPL": {"sma_50": 180.5, "ema_50": 182.1, "volatility": 1.9},
        "GOOG": {"sma_50": 140.8, "ema_50": 141.3, "volatility": 2.1},
        "TSLA": {"sma_50": 215.7, "ema_50": 218.2, "volatility": 3.5},
    }
    return quant_metrics.get(stock_symbol, {})

In [6]:
# 4. Investment Strategy Decision
def investment_strategy(stock_symbol: str, market_data: dict, sentiment: dict, quant: dict) -> str:
    """Analyze data and generate buy/sell/hold recommendation."""
    if not market_data or not sentiment or not quant:
        return "Not enough data for recommendation."

    decision = "Hold"
    if market_data["pe_ratio"] < 30 and sentiment["news_sentiment"] == "Positive" and quant["volatility"] < 2:
        decision = "Buy"
    elif market_data["pe_ratio"] > 35 or sentiment["news_sentiment"] == "Negative":
        decision = "Sell"

    return f"Recommended Action for {stock_symbol}: {decision}"

## Define the react and supervisor agents

In [12]:
### --- CREATE AGENTS --- ###
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
# Market Data Agent
market_data_expert = create_react_agent(
    model=model,
    tools=[fetch_market_data],
    name="market_data_expert",
    prompt="You are an expert in stock market data. Fetch stock data when requested."
)

# Sentiment Analysis Agent
sentiment_expert = create_react_agent(
    model=model,
    tools=[analyze_sentiment],
    name="sentiment_expert",
    prompt="You analyze financial news and social media sentiment for stock symbols."
)

# Quantitative Analysis Agent
quant_expert = create_react_agent(
    model=model,
    tools=[compute_quant_metrics],
    name="quant_expert",
    prompt="You analyze stock price trends, moving averages, and volatility metrics."
)

# Investment Strategy Agent
strategy_expert = create_react_agent(
    model=model,
    tools=[investment_strategy],
    name="strategy_expert",
    prompt="You make investment recommendations based on market, sentiment, and quant data."
)

### --- SUPERVISOR AGENT --- ###
# Create supervisor workflow
workflow = create_supervisor(
    [ market_data_expert,sentiment_expert,quant_expert , strategy_expert],
    model=model,
    prompt=(
       """You are a financial market supervisor managing four expert agents: market data, sentiment, 
        quantitative analysis, and investment strategy. For stock queries, use market_data_expert. 
        For news/social sentiment, use sentiment_expert. For stock price analysis, use quant_expert. 
        For final investment recommendations, use strategy_expert."""
    )
)

# Compile and run
app = workflow.compile()


## Invoke and print response

In [20]:
### --- RUN THE SYSTEM --- ###
stock_query = {
    "messages": [
        {"role": "user", "content": "What is the investment recommendation for AAPL?"}
    ]
}

# Execute query
result = app.invoke(stock_query)

#print(result['messages'][-1].content)
for m in result["messages"]:
    m.pretty_print()


================================ Human Message =================================

What is the investment recommendation for AAPL?
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_strategy_expert (chatcmpl-tool-07febcc7a70542f2ab6eb30d39f38fc3)
 Call ID: chatcmpl-tool-07febcc7a70542f2ab6eb30d39f38fc3
  Args:
================================= Tool Message =================================
Name: transfer_to_strategy_expert

Successfully transferred to strategy_expert
================================== Ai Message ==================================
Name: strategy_expert

To provide an accurate recommendation, I need to analyze the current market data, sentiment, and quantitative data for AAPL. Here's a hypothetical analysis:

1. Market Data:
   - Current price: $150.78 per share
   - 52-week high: $157.26
   - 52-week low: $103.86
   - Earnings per share (EPS): $5.03
   - Dividend yield: 0.65%

2. Sentiment:
   - Ana